In [48]:
import pandas as pd
import urllib
import mysql.connector
from mysql.connector import connection
from tqdm.notebook import trange, tqdm
import re
from sqlalchemy import create_engine
import datetime

In [19]:
from unidecode import unidecode

#### BANCO AWS

In [51]:
con_mysql = connection.MySQLConnection()

In [52]:
df_mysql = pd.read_sql_query("""SELECT COD_CLIENTE, BASE 
                                FROM bradvisors.base_clientes;""", con = con_mysql)

df_mysql['COD_CLIENTE'] = df_mysql['COD_CLIENTE'].astype('string')
df_mysql['BASE'] = df_mysql['BASE'].astype('string')
df_mysql['BASE_LOWER'] = df_mysql['BASE'].astype('string').str.lower()
df_mysql = df_mysql[~df_mysql['BASE'].isna()]
df_mysql['BASE_LOWER'] = [unidecode(x) for x in df_mysql['BASE_LOWER']]

df_mysql

C:\Users\RodrigoBraga\AppData\Local\Temp\ipykernel_41864\1168884411.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_mysql = pd.read_sql_query("""SELECT COD_CLIENTE, BASE


,COD_CLIENTE,BASE,BASE_LOWER
3,4,BRA Peu,bra peu
4,16,Pupi,pupi
5,18,Pupi,pupi
6,19,Pedro Cavalcante,pedro cavalcante
7,35,BRA,bra
...,...,...,...
84840,22362770,BRA AlgoTrading,bra algotrading
84841,32698429,BRA Wanderley,bra wanderley
84842,34673461,BRA Tráfego Pago,bra trafego pago
84843,63551109,BRA Marketing Inbound,bra marketing inbound


#### BANCO AZURE

In [53]:
# Configurações do banco de dados
db_host = 'nomos-sqlazure01.database.windows.net'
db_port = 
db_name = 
db_user = 
db_pass = 


In [54]:
# Função para criar a engine do SQLAlchemy
def create_db_engine(db_host, db_port, db_name, db_user, db_pass):
    connection_string = f"Driver={{ODBC Driver 17 for SQL Server}};" \
                        f"Server={db_host},{db_port};" \
                        f"Database={db_name};" \
                        f"UID={db_user};" \
                        f"PWD={db_pass};"
    quoted = urllib.parse.quote_plus(connection_string)
    engine = create_engine(f"mssql+pyodbc:///?odbc_connect={quoted}", fast_executemany=True)
    return engine

In [55]:
# Criar engine do SQLAlchemy para o banco de dados
engine_sqlserver = create_db_engine(db_host, db_port, db_name, db_user, db_pass)

In [56]:
df_sqlserver = pd.read_sql("""SELECT ID_ORIGEM, NOME_ORIGEM
                              FROM tb_origens;""", engine_sqlserver)

df_sqlserver['ID_ORIGEM'] = df_sqlserver['ID_ORIGEM'].astype('string')
df_sqlserver['NOME_ORIGEM'] = df_sqlserver['NOME_ORIGEM'].astype('string')
df_sqlserver['NOME_ORIGEM_LOWER'] = df_sqlserver['NOME_ORIGEM'].astype('string').str.lower()
df_sqlserver = df_sqlserver[~df_sqlserver['NOME_ORIGEM'].isna()]
df_sqlserver['NOME_ORIGEM_LOWER'] = [unidecode(x) for x in df_sqlserver['NOME_ORIGEM_LOWER']]

df_sqlserver

,ID_ORIGEM,NOME_ORIGEM,NOME_ORIGEM_LOWER
0,1,Ainda sem base,ainda sem base
1,2,BRA - FAMERSON MIKE,bra - famerson mike
2,3,BRA Acordo Certo,bra acordo certo
3,4,BRA Ale Bullckers,bra ale bullckers
4,5,BRA Ale Diego Ribeiro,bra ale diego ribeiro
...,...,...,...
366,398,NOMOS Octávio Capitalistas EIX,nomos octavio capitalistas eix
367,399,NOMOS Octávio Malka,nomos octavio malka
368,400,NOMOS Octávio Blessed Company,nomos octavio blessed company
369,401,NOMOS Pedro Muger MKT,nomos pedro muger mkt


In [57]:
# 84845 mysql
# 371 sql server

In [58]:
#### MERGE PARA OBTER REQUSITOS

In [59]:
df_final = pd.merge(df_mysql, df_sqlserver, left_on='BASE_LOWER', right_on='NOME_ORIGEM_LOWER', how = 'left')
df_final = df_final[['COD_CLIENTE', 'BASE', 'ID_ORIGEM', 'NOME_ORIGEM']]
df_final

,COD_CLIENTE,BASE,ID_ORIGEM,NOME_ORIGEM
0,4,BRA Peu,<NA>,<NA>
1,16,Pupi,<NA>,<NA>
2,18,Pupi,<NA>,<NA>
3,19,Pedro Cavalcante,<NA>,<NA>
4,35,BRA,<NA>,<NA>
...,...,...,...,...
85272,22362770,BRA AlgoTrading,11,BRA AlgoTrading
85273,32698429,BRA Wanderley,265,BRA Wanderley
85274,34673461,BRA Tráfego Pago,263,BRA Tráfego Pago
85275,63551109,BRA Marketing Inbound,108,BRA Marketing Inbound


#### pulo do gato

In [60]:
df_clientes_sqlserver = pd.read_sql("""SELECT CODIGO_CLIENTE
                              FROM tb_clientes
                              WHERE ID_ORIGEM IS NULL;""", engine_sqlserver)
df_clientes_sqlserver['CODIGO_CLIENTE'] = df_clientes_sqlserver['CODIGO_CLIENTE'].astype('string')

df_clientes_sqlserver

,CODIGO_CLIENTE
0,37308
1,37317
2,37456
3,37687
4,37700
...,...
135747,11680270
135748,2370267
135749,11673676
135750,2108403


In [61]:
df_final_final = pd.merge(df_final, df_clientes_sqlserver, left_on='COD_CLIENTE', right_on='CODIGO_CLIENTE', how = 'inner')
df_final_final

,COD_CLIENTE,BASE,ID_ORIGEM,NOME_ORIGEM,CODIGO_CLIENTE
0,19,Pedro Cavalcante,<NA>,<NA>,19
1,51,BRA,<NA>,<NA>,51
2,52,BRA,<NA>,<NA>,52
3,57,Rafael Vianna,<NA>,<NA>,57
4,443,Beto,<NA>,<NA>,443
...,...,...,...,...,...
80582,11667583,BRA AJUSTAR,<NA>,<NA>,11667583
80583,11671147,BRA AJUSTAR,<NA>,<NA>,11671147
80584,11672929,BRA AJUSTAR,<NA>,<NA>,11672929
80585,11672969,BRA AJUSTAR,<NA>,<NA>,11672969


In [64]:
df_final_final = df_final_final[~df_final_final['ID_ORIGEM'].isna()]
df_final_final

,COD_CLIENTE,BASE,ID_ORIGEM,NOME_ORIGEM,CODIGO_CLIENTE
6,753,BRA Famerson Marketing,83,BRA Famerson Marketing,753
7,871,BRA Peu Investidor Bot,191,BRA Peu Investidor Bot,871
9,5259,João Victor,269,João Victor,5259
15,9332,BRA AlgoTrading,11,BRA AlgoTrading,9332
17,12040,BRA Caio Marketing,52,BRA Caio Marketing,12040
...,...,...,...,...,...
80436,11606845,BRA Ricardo Brasil SQUAD,242,BRA Ricardo Brasil SQUAD,11606845
80459,11612594,BRA Ricardo Brasil SQUAD,242,BRA Ricardo Brasil SQUAD,11612594
80482,11622874,BRA Ricardo Brasil SQUAD,242,BRA Ricardo Brasil SQUAD,11622874
80497,11626072,BRA Ricardo Brasil SQUAD,242,BRA Ricardo Brasil SQUAD,11626072


In [65]:
conn_sqlserver = engine_sqlserver.connect()

In [66]:
#CRIANDO O CURSOR
cursor_sqlserver = conn_sqlserver.connection.cursor()

In [70]:
for index, row in tqdm(df_final_final[1:1000].iterrows()):
    codigo_cliente = row['CODIGO_CLIENTE']
    id_origem = row['ID_ORIGEM']
    print('########################################')
    print('ID_ORIGEM: ', id_origem)
    print('Cogigo Cliente: ', codigo_cliente)
    
    # Executar a query de atualização
    query = f"UPDATE tb_clientes SET ID_ORIGEM = {id_origem} WHERE CODIGO_CLIENTE = {codigo_cliente}"
    cursor_sqlserver.execute(query)    

0it [00:00, ?it/s]

########################################
ID_ORIGEM:  191
Cogigo Cliente:  871
########################################
ID_ORIGEM:  269
Cogigo Cliente:  5259
########################################
ID_ORIGEM:  11
Cogigo Cliente:  9332
########################################
ID_ORIGEM:  52
Cogigo Cliente:  12040
########################################
ID_ORIGEM:  269
Cogigo Cliente:  15317
########################################
ID_ORIGEM:  299
Cogigo Cliente:  15404
########################################
ID_ORIGEM:  200
Cogigo Cliente:  17224
########################################
ID_ORIGEM:  91
Cogigo Cliente:  17278
########################################
ID_ORIGEM:  269
Cogigo Cliente:  17590
########################################
ID_ORIGEM:  189
Cogigo Cliente:  19449
########################################
ID_ORIGEM:  269
Cogigo Cliente:  19804
########################################
ID_ORIGEM:  269
Cogigo Cliente:  20012
########################################
ID_ORI

########################################
ID_ORIGEM:  16
Cogigo Cliente:  57675
########################################
ID_ORIGEM:  263
Cogigo Cliente:  58476
########################################
ID_ORIGEM:  269
Cogigo Cliente:  58480
########################################
ID_ORIGEM:  269
Cogigo Cliente:  58933
########################################
ID_ORIGEM:  111
Cogigo Cliente:  59323
########################################
ID_ORIGEM:  269
Cogigo Cliente:  59335
########################################
ID_ORIGEM:  269
Cogigo Cliente:  59521
########################################
ID_ORIGEM:  269
Cogigo Cliente:  59747
########################################
ID_ORIGEM:  269
Cogigo Cliente:  60833
########################################
ID_ORIGEM:  269
Cogigo Cliente:  60833
########################################
ID_ORIGEM:  269
Cogigo Cliente:  61820
########################################
ID_ORIGEM:  269
Cogigo Cliente:  61820
########################################


########################################
ID_ORIGEM:  269
Cogigo Cliente:  85558
########################################
ID_ORIGEM:  242
Cogigo Cliente:  85945
########################################
ID_ORIGEM:  242
Cogigo Cliente:  85945
########################################
ID_ORIGEM:  91
Cogigo Cliente:  86008
########################################
ID_ORIGEM:  269
Cogigo Cliente:  86012
########################################
ID_ORIGEM:  269
Cogigo Cliente:  86099
########################################
ID_ORIGEM:  245
Cogigo Cliente:  86946
########################################
ID_ORIGEM:  269
Cogigo Cliente:  87967
########################################
ID_ORIGEM:  269
Cogigo Cliente:  87967
########################################
ID_ORIGEM:  269
Cogigo Cliente:  87971
########################################
ID_ORIGEM:  263
Cogigo Cliente:  88885
########################################
ID_ORIGEM:  11
Cogigo Cliente:  89071
########################################
I

########################################
ID_ORIGEM:  163
Cogigo Cliente:  128865
########################################
ID_ORIGEM:  269
Cogigo Cliente:  129446
########################################
ID_ORIGEM:  269
Cogigo Cliente:  129446
########################################
ID_ORIGEM:  269
Cogigo Cliente:  130189
########################################
ID_ORIGEM:  263
Cogigo Cliente:  132080
########################################
ID_ORIGEM:  87
Cogigo Cliente:  133529
########################################
ID_ORIGEM:  52
Cogigo Cliente:  133894
########################################
ID_ORIGEM:  269
Cogigo Cliente:  136880
########################################
ID_ORIGEM:  269
Cogigo Cliente:  137453
########################################
ID_ORIGEM:  269
Cogigo Cliente:  138195
########################################
ID_ORIGEM:  269
Cogigo Cliente:  139054
########################################
ID_ORIGEM:  269
Cogigo Cliente:  139054
##############################

...

########################################
ID_ORIGEM:  269
Cogigo Cliente:  228525
########################################
ID_ORIGEM:  269
Cogigo Cliente:  228803
########################################
ID_ORIGEM:  269
Cogigo Cliente:  228803
########################################
ID_ORIGEM:  27
Cogigo Cliente:  229728
########################################
ID_ORIGEM:  269
Cogigo Cliente:  229835
########################################
ID_ORIGEM:  269
Cogigo Cliente:  229835
########################################
ID_ORIGEM:  16
Cogigo Cliente:  230243
########################################
ID_ORIGEM:  263
Cogigo Cliente:  230355
########################################
ID_ORIGEM:  248
Cogigo Cliente:  230667
########################################
ID_ORIGEM:  267
Cogigo Cliente:  230667
########################################
ID_ORIGEM:  254
Cogigo Cliente:  230681
########################################
ID_ORIGEM:  10
Cogigo Cliente:  230695
###############################

########################################
ID_ORIGEM:  236
Cogigo Cliente:  251566
########################################
ID_ORIGEM:  269
Cogigo Cliente:  251618
########################################
ID_ORIGEM:  269
Cogigo Cliente:  251618
########################################
ID_ORIGEM:  263
Cogigo Cliente:  252428
########################################
ID_ORIGEM:  200
Cogigo Cliente:  253086
########################################
ID_ORIGEM:  269
Cogigo Cliente:  253277
########################################
ID_ORIGEM:  353
Cogigo Cliente:  253642
########################################
ID_ORIGEM:  269
Cogigo Cliente:  254272
########################################
ID_ORIGEM:  109
Cogigo Cliente:  254596
########################################
ID_ORIGEM:  110
Cogigo Cliente:  254596
########################################
ID_ORIGEM:  245
Cogigo Cliente:  254634
########################################
ID_ORIGEM:  269
Cogigo Cliente:  255381
############################

########################################
ID_ORIGEM:  204
Cogigo Cliente:  284955
########################################
ID_ORIGEM:  269
Cogigo Cliente:  286044
########################################
ID_ORIGEM:  269
Cogigo Cliente:  286086
########################################
ID_ORIGEM:  269
Cogigo Cliente:  286086
########################################
ID_ORIGEM:  70
Cogigo Cliente:  286167
########################################
ID_ORIGEM:  93
Cogigo Cliente:  286337
########################################
ID_ORIGEM:  269
Cogigo Cliente:  286750
########################################
ID_ORIGEM:  269
Cogigo Cliente:  286750
########################################
ID_ORIGEM:  10
Cogigo Cliente:  286799
########################################
ID_ORIGEM:  269
Cogigo Cliente:  286943
########################################
ID_ORIGEM:  269
Cogigo Cliente:  286943
########################################
ID_ORIGEM:  99
Cogigo Cliente:  286981
################################

########################################
ID_ORIGEM:  102
Cogigo Cliente:  301282
########################################
ID_ORIGEM:  269
Cogigo Cliente:  301402
########################################
ID_ORIGEM:  263
Cogigo Cliente:  301639
########################################
ID_ORIGEM:  245
Cogigo Cliente:  301823
########################################
ID_ORIGEM:  104
Cogigo Cliente:  302240
########################################
ID_ORIGEM:  269
Cogigo Cliente:  302648
########################################
ID_ORIGEM:  255
Cogigo Cliente:  302792
########################################
ID_ORIGEM:  263
Cogigo Cliente:  302870
########################################
ID_ORIGEM:  238
Cogigo Cliente:  302872
########################################
ID_ORIGEM:  269
Cogigo Cliente:  302992
########################################
ID_ORIGEM:  269
Cogigo Cliente:  302992
########################################
ID_ORIGEM:  16
Cogigo Cliente:  303212
#############################

########################################
ID_ORIGEM:  99
Cogigo Cliente:  322055
########################################
ID_ORIGEM:  269
Cogigo Cliente:  322121
########################################
ID_ORIGEM:  269
Cogigo Cliente:  322121
########################################
ID_ORIGEM:  125
Cogigo Cliente:  322165
########################################
ID_ORIGEM:  269
Cogigo Cliente:  322181
########################################
ID_ORIGEM:  269
Cogigo Cliente:  322480
########################################
ID_ORIGEM:  263
Cogigo Cliente:  322511
########################################
ID_ORIGEM:  269
Cogigo Cliente:  322516
########################################
ID_ORIGEM:  11
Cogigo Cliente:  322776
########################################
ID_ORIGEM:  269
Cogigo Cliente:  322780
########################################
ID_ORIGEM:  269
Cogigo Cliente:  322780
########################################
ID_ORIGEM:  100
Cogigo Cliente:  322790
##############################

In [69]:
#Confirmar as atualizações
cursor_sqlserver.commit()

In [ ]:
# Fechar o cursor
cursor.close()

In [ ]:
try:
    # Iniciar a transação
    #conn.begin()

    # Iterar sobre os dados do dataframe e realizar as atualizações
    for index, row in df_final.iterrows():
        codigo_cliente = row['CODIGO_CLIENTE']
        id_origem = row['ID_ORIGEM']
        print('########################################')
        print('ID_ORIGEM: ', id_origem)
        print('Cogigo Cliente: ', codigo_cliente)

#         # Executar a query de atualização
#         query = f"UPDATE tb_clientes SET ID_ORIGEM = {id_origem} WHERE CODIGO_CLIENTE = {codigo_cliente}"
#         cursor.execute(query)

#     # Confirmar as atualizações
#     conn.commit()

#     # Fechar o cursor
#     cursor.close()

#     # Exibir mensagem de sucesso
#     print("As atualizações foram realizadas com sucesso.")

# except Exception as e:
#     # Em caso de erro, desfazer as alterações
#     conn.rollback()

#     # Exibir mensagem de erro
#     print("Ocorreu um erro durante as atualizações:", str(e))

# finally:
#     # Fechar a conexão com o banco de dados
#     conn.close()